# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# TODO: Import the necessary libs
# For example: 
import os
import chromadb
from chromadb.utils import embedding_functions
from chromadb.api.models.Collection import Collection

from lib.agents import Agent, AgentState
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from dotenv import load_dotenv
from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Resource
from typing import TypedDict, List

from lib.evaluation import TestCase, AgentEvaluator, EvaluationResult
from lib.state_machine import Run
import time
from typing import List, Dict

from pydantic import BaseModel

from datetime import datetime
from tavily import TavilyClient

In [3]:
# TODO: Load environment variables
# load_dotenv()

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# all variables are loaded from .env file in gitignore i used my private keys
load_dotenv()

True

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [4]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
chroma_client = chromadb.PersistentClient(path="chromadb")      

# Use the same embedding function 
embedding_fn = embedding_functions.OpenAIEmbeddingFunction()
collection = chroma_client.get_collection("udaplay", embedding_function=embedding_fn)

@tool
def retrieve_game(query: str) -> str:
    """
    Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry. 

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    # Search the collection for relevant games
    results = collection.query(
        query_texts=[query],
        n_results=5  # Get top 5 most relevant results
    )
    
    # Format the results for 
    if not results['documents'][0]:
        return "No games found matching your query."
    
    formatted_results = []
    for i in range(len(results['documents'][0])):
        metadata = results['metadatas'][0][i]
        document = results['documents'][0][i]
        
        game_info = {
            "Platform": metadata.get("Platform", "Unknown"),
            "Name": metadata.get("Name", "Unknown"),
            "YearOfRelease": metadata.get("YearOfRelease", "Unknown"),
            "Description": metadata.get("Description", "No description available"),
            "Genre": metadata.get("Genre", "Unknown"),
            "Publisher": metadata.get("Publisher", "Unknown")
        }
        formatted_results.append(game_info)
    
    return str(formatted_results)

In [5]:
# Test the retrieve_game tool (ADDITONAL TEST SCRIPT) Works!
test_query = "which games have a racing characrer?"
print("Testing retrieve_game tool with query:", test_query)
print("=" * 50)

result = retrieve_game(test_query)
print(result)

Testing retrieve_game tool with query: which games have a racing characrer?
[{'Platform': 'PlayStation 3', 'Name': 'Gran Turismo 5', 'YearOfRelease': 2010, 'Description': 'A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.', 'Genre': 'Racing', 'Publisher': 'Sony Computer Entertainment'}, {'Platform': 'PlayStation 1', 'Name': 'Gran Turismo', 'YearOfRelease': 1997, 'Description': 'A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.', 'Genre': 'Racing', 'Publisher': 'Sony Computer Entertainment'}, {'Platform': 'Nintendo Switch', 'Name': 'Mario Kart 8 Deluxe', 'YearOfRelease': 2017, 'Description': 'An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics.', 'Genre': 'Racing', 'Publisher': 'Nintendo'}, {'Platform': 'Nintendo 64', 'Name': 'Super Mario 64', 'YearOfRelease': 1996, 'Description': "A groundbreaking 3D platformer 

In [ ]:
# (ADDITIONAL TEST SCRIPT) Works!  used the evaluator to test the agents output!
agent = Agent(
    model_name="gpt-4o-mini",
    tools=[retrieve_game],
    instructions="You can bring insights about a game dataset based on users questions",
)

evaluator = AgentEvaluator()

In [ ]:
# (ADDITIONAL TEST SCRIPT) Works!  used the evaluator to test the agents output!
test_cases = [
    TestCase(
        id="game_query_1",
        description="FInd a game with a racing character",
        user_query="I am interested in games with a racing character, do you know any?",
        expected_tools=["retrieve_game"],
        reference_answer="Gran Turismo 5",
        max_steps=4
    ),
]

for test_case in test_cases:
    print(f"\n=== Evaluating Test Case: {test_case.id} ===")
    print(f"Description: {test_case.description}")
    print(f"Query: {test_case.user_query}")
    
    # Run the agent
    start_time = time.time()
    print("\nWorkflow:")
    agent.memory.reset()
    run_object:Run = agent.invoke(test_case.user_query)
    execution_time = time.time() - start_time
    
    # Get final state and response
    final_state:AgentState = run_object.get_final_state()
    if final_state and final_state.get("messages"):
        # Find the last AI message as the final response
        final_response = ""
        for msg in reversed(final_state["messages"]):
            if isinstance(msg, AIMessage) and msg.content:
                final_response = msg.content
                break
        
        total_tokens = final_state.get("total_tokens", 0)
        
        # Evaluate using all three methods
        print("\n--- Black Box (Final Response) Evaluation ---")
        black_box_eval:EvaluationResult = evaluator.evaluate_final_response(
            test_case, final_response, execution_time, total_tokens
        )
        print(f"Overall Score: {black_box_eval.overall_score:.2f}")
        print(f"Task Completed: {black_box_eval.task_completion.task_completed}")
        print(f"Feedback: {black_box_eval.feedback}")
        
        print("\n--- Single Step Evaluation ---")
        step_eval:EvaluationResult = evaluator.evaluate_single_step(
            final_state["messages"], test_case.expected_tools
        )
        print(f"Overall Score: {step_eval.overall_score:.2f}")
        print(f"Correct Tool Selected: {step_eval.tool_interaction.correct_tool_selected}")
        print(f"Feedback: {step_eval.feedback}")
        
        print("\n--- Trajectory Evaluation ---")
        traj_eval:EvaluationResult = evaluator.evaluate_trajectory(test_case, run_object)
        print(f"Overall Score: {traj_eval.overall_score:.2f}")
        print(f"Steps Taken: {traj_eval.task_completion.steps_taken}")
        print(f"Total Tokens: {traj_eval.system_metrics.total_tokens}")
        print(f"Execution Time: {traj_eval.system_metrics.execution_time:.2f}s")
        print(f"Estimated Cost: ${traj_eval.system_metrics.cost_estimate:.6f}")
        print(f"Feedback: {traj_eval.feedback}")
        
    else:
        print("ERROR: No final state or messages found")


=== Evaluating Test Case: game_query_1 ===
Description: FInd a game with a racing character
Query: I am interested in games with a racing character, do you know any?

Workflow:
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

--- Black Box (Final Response) Evaluation ---
Overall Score: 1.00
Task Completed: True
Feedback: The agent successfully provided a list of games that feature a racing character, which directly answers the user's query. The format is clear and organized, listing the games with relevant details such as platform, year of release, and description. Additionally, the agent followed the implicit instruction to provide multiple examples and offered to assist further if the user has specific preferences, demonstrating attentiveness to the user's needs.

--- Single S

#### Evaluate Retrieval Tool

In [ ]:
# TODO: Create evaluate_retrieval tool
class EvaluationReport(BaseModel):
    """Evaluation report for document retrieval quality"""
    useful: bool
    description: str

# Create LLM instance for evaluation
evaluator_llm = LLM(model="gpt-4o-mini", temperature=0.1)

@tool
def evaluate_retrieval(question: str, retrieved_docs: str) -> str:
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question. 
    args: 
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """
    
    # System message for the LLM judge
    system_message = """You are a strict document evaluator you only evaluate NOT MAKE ANY INTERPRETATIONS. Your task is to evaluate if the documents are enough to can respond to the users query accurately.

Be very strict here in the following in your evaluation
1. Do the documents contain the information needed to answer the question fully?
2. Is the information relevant and directly usefull?
3. Are there any informations gaps that would prevent a complete answer?
4. Is the quality of information sufficient?

Provide a detailed explanation so it's possible to take an action to accept the documents or search for additional information."""

    # User prompt for evaluation
    user_prompt = f"""
Question: {question}

Retrieved Documents: {retrieved_docs}

Please evaluate these retrieved documents are sufficient to answer the user question.

Return your evaluation in the following JSON format:
{{
    "useful": true/false,
    "description": "detailed explanation of your evaluation, including what information is present, what might be missing, and whether the documents can adequately answer the question"
}}
"""

    try:
        # Create messages for the LLM
        messages = [
            SystemMessage(content=system_message),
            UserMessage(content=user_prompt)
        ]
        
        # Get evaluation from LLM
        response = evaluator_llm.invoke(messages)
        
        # Try to parse as JSON and validate with Pydantic
        import json
        try:
            result_dict = json.loads(response.content)
            evaluation_report = EvaluationReport(**result_dict)
            return f"Useful: {evaluation_report.useful}\nDescription: {evaluation_report.description}"
        except (json.JSONDecodeError, ValueError) as e:
            # If parsing fails, return the raw response
            return f"parsing failed. Raw response: {response.content}"
            
    except Exception as e:
        return f"Error during evaluation: {str(e)}"


# Test the tool (ADDITIONAL TEST SCRIPT) Works!
if __name__ == "__main__":
    # Example test
    test_question = "Can you recommend a racing?"
    test_docs = """[{'Platform': 'PlayStation 3', 'Name': 'Gran Turismo 5', 'YearOfRelease': 2010, 'Description': 'A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.', 'Genre': 'Racing', 'Publisher': 'Sony Computer Entertainment'}, {'Platform': 'PlayStation 1', 'Name': 'Gran Turismo', 'YearOfRelease': 1997, 'Description': 'A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.', 'Genre': 'Racing', 'Publisher': 'Sony Computer Entertainment'}, {'Platform': 'Nintendo Switch', 'Name': 'Mario Kart 8 Deluxe', 'YearOfRelease': 2017, 'Description': 'An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics.', 'Genre': 'Racing', 'Publisher': 'Nintendo'}, {'Platform': 'Nintendo 64', 'Name': 'Super Mario 64', 'YearOfRelease': 1996, 'Description': "A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.", 'Genre': 'Platformer', 'Publisher': 'Nintendo'}, {'Platform': 'PlayStation 2', 'Name': 'Grand Theft Auto: San Andreas', 'YearOfRelease': 2004, 'Description': "An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson.", 'Genre': 'Action-adventure', 'Publisher': 'Rockstar Games'}]"""
    
    result = evaluate_retrieval(test_question, test_docs)
    print("Test Result:")
    print(result)

Test Result:
Useful: True
Description: The retrieved documents contain relevant information that directly addresses the user's query about recommending a racing game. Specifically, there are three racing games listed: 'Gran Turismo 5' (2010), 'Gran Turismo' (1997), and 'Mario Kart 8 Deluxe' (2017). Each entry includes the platform, name, year of release, description, genre, and publisher, which provides a comprehensive overview of each game. The information is sufficient to recommend these titles to the user. However, there is a minor gap in that the documents do not provide a comparative analysis or personal recommendations based on user preferences, but they still adequately fulfill the basic requirement of suggesting racing games. Overall, the quality of information is sufficient for the purpose of answering the user's question.


#### Game Web Search Tool

In [14]:
# TODO: Create game_web_search tool
# INVOKE the Tavily client to search the web
api_key_tavily = os.getenv("TAVILY_API_KEY")
client = TavilyClient(api_key=api_key_tavily)

In [15]:
# Test the Tavily client with a search query
result = client.search("Which game everyone is waiting for in 2025?")
result

{'query': 'Which game everyone is waiting for in 2025?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.reddit.com/r/gaming/comments/1hw0e7x/which_2025_game_are_you_most_looking_forward_to/',
   'title': 'Which 2025 game are you most looking forward to? - Reddit',
   'content': "Either Doom: The Dark Ages or Metal Gear Solid Delta for me personally. I'm not saying GTA 6 as I think it will probably get delayed.",
   'score': 0.8351749,
   'raw_content': None},
  {'url': 'https://www.gamesradar.com/video-game-release-dates/',
   'title': 'New games 2025 release schedule | GamesRadar+',
   'content': "Subscribe from just £3 Takes you closer to the games, movies and TV you love Try a single issue or save on a subscription Issues delivered straight to your door or device From$12 View Trending GTA 6 trailer 2 Upcoming Switch 2 games FGS Live Latam The Duskbloods Games New games 2025 and beyond: The biggest video game release dates for PS5, Xbo

In [16]:
# Craete the web_search tool to use for agents
@tool
def web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web using Tavily API
    
    """
    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    
    return formatted_results

In [17]:
# create the list for the tools
tools = [web_search, retrieve_game, evaluate_retrieval]

### Agent

In [19]:
# TODO: Create your Agent abstraction using StateMachine

#Solutions as an agent!

game_advisor_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
            "You are a game advisor agent. Your task is to provide insights about games based on user questions. "
            "You can use the tools provided to search the web, retrieve game information, and evaluate the quality of your responses. "
            "Make sure to use the tools effectively to gather information and provide accurate answers. "
            "Always start with searching your own database for relevant games, and if necessary, use the web search if the evaluation tool indicates that the retrieved documents are not sufficient. "
    ),
    tools=tools
)

In [ ]:
# Test script for one usecase.

# delete memory as it is session based
game_advisor_agent.memory.reset()
# 1. Ask your question
result = game_advisor_agent.invoke("When were Pokémon Gold and Silver released?")
# 2. Get the answer with reasoning and tool usage
final_state = result.get_final_state()

# Show agent reasoning and tool usage
print("Agent Reasoning & Tool Usage:")
messages = final_state.get("messages", [])
tools_used = []

for msg in messages:
    if isinstance(msg, AIMessage) and hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tool_call in msg.tool_calls:
            tool_name = tool_call.function.name
            tools_used.append(tool_name)
            print(f"Using tool: {tool_name}")
            print(f"Arguments: {tool_call.function.arguments}")
    elif isinstance(msg, ToolMessage):
        result_preview = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
        print(f"Tool result: {result_preview}")

# Show final answer
print("Final Answer:")
final_answer_found = False
for msg in reversed(final_state["messages"]):
    if isinstance(msg, AIMessage) and msg.content:
        # Check if this is a final answer (not a tool call message)
        if not hasattr(msg, 'tool_calls') or not msg.tool_calls:
            print(msg.content)
            final_answer_found = True
            break

if not final_answer_found:
    print("No final answer found in messages")

print(f"Tools used: {', '.join(set(tools_used)) if tools_used else 'None'}")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Agent Reasoning & Tool Usage:
Using tool: retrieve_game
Arguments: {"query":"Pokémon Gold and Silver release date"}
Tool result: "[{'Platform': 'Game Boy Color', 'Name': 'Pok\u00e9mon Gold and Silver', 'YearOfRelease': 1999, 'Des...
Final Answer:
Pokémon Gold and Silver were released for the Game Boy Color in 1999. These games are known as the second generation of Pokémon games, introducing new regions, Pokémon, and gameplay mechanics.
Tools used: retrieve_game


In [21]:
# Solution AS a complete workflow, StateMachine that will use the tools and Agent in a workflow for more controll!! (i like this more as it is less error aprun)
class GameResearchState(TypedDict):
    user_query: str
    retrieved_docs: str
    evaluation_result: str
    web_search_result: str
    final_answer: str
    confidence_score: float
    sources_used: List[str]


def query_vector_db(state: GameResearchState) -> Dict:
    """Step 1: Ask the vector database for relevant games"""
    print("Searcing my database for information about the question :)")
    
    retrieved_docs = retrieve_game(state["user_query"])
    sources_used = state.get("sources_used", []) + ["vector_database"]
    
    print(f"Found documents")
    return {
        "retrieved_docs": retrieved_docs,
        "sources_used": sources_used
    }

def evaluate_documents(state: GameResearchState) -> Dict:
    """Step 2: Evaluate if retrieved documents are sufficient"""
    print(" EValuating the quality of the answer from the db. Hope its GOOD")
    
    if state["retrieved_docs"] == "No games found matching your query.":
        evaluation_result = "Useful: False\nDescription: No documents were retrieved from the vector database."
        confidence_score = 0.0
    else:
        evaluation_result = evaluate_retrieval(state["user_query"], state["retrieved_docs"])
        # Extract confidence score from evaluation
        confidence_score = 0.8 if "Useful: True" in evaluation_result else 0.3
    
    print(f"Evaluation complete. Confidence: {confidence_score}")
    return {
        "evaluation_result": evaluation_result,
        "confidence_score": confidence_score
    }

def should_search_web(state: GameResearchState) -> str:
    """Decision point: Should we search the web?"""
    return "web_search" if state["confidence_score"] < 0.7 else "generate_answer"

def search_web_step(state: GameResearchState) -> Dict:
    """Step 3: Search the web if needed"""
    print("Searching the web for additional information...")
    
    web_result = web_search(state["user_query"])
    sources_used = state.get("sources_used", []) + ["web_search"]
    confidence_score = min(state["confidence_score"] + 0.4, 1.0)
    
    print("Web search complete")
    return {
        "web_search_result": str(web_result),
        "sources_used": sources_used,
        "confidence_score": confidence_score
    }

def generate_final_answer(state: GameResearchState) -> Dict:
    """Step 4: Generate comprehensive final answer"""
    print("Generating my final answer...")
    
    # Create a comprehensive prompt for the LLM
    context_parts = []
    
    if state["retrieved_docs"] and state["retrieved_docs"] != "No games found matching your query.":
        context_parts.append(f"Vector Database Results:\n{state['retrieved_docs']}")
    
    if state.get("web_search_result"):
        context_parts.append(f"Web Search Results:\n{state['web_search_result']}")
    
    context = "\n\n".join(context_parts)
    
    final_prompt = f"""
    Based on the following information, provide a answer to the user's question: "{state['user_query']}"

    Available Information:
    {context}

    Please provide a well structured answer that answers the users question directly, if you could not find the answer explain why.

    Sources used: {', '.join(state.get('sources_used', []))}
    Confidence level: {state['confidence_score']}
    """
    
    # Use the LLM to generate the final answer
    llm = LLM(model="gpt-4o-mini", temperature=0.3)
    messages = [
        SystemMessage(content="You are an expert game advisor working at our game store. Provide comprehensive, accurate information about video games."),
        UserMessage(content=final_prompt)
    ]
    
    response = llm.invoke(messages)
    
    print("Final answer generated")
    return {"final_answer": response.content}

# Create the StateMachine using the actual API
game_research_sm = StateMachine(GameResearchState)

# Add all steps
entry_step = EntryPoint()
query_step = Step("query_db", query_vector_db)
evaluate_step = Step("evaluate", evaluate_documents)
web_search_step = Step("web_search", search_web_step)
answer_step = Step("generate_answer", generate_final_answer)
termination_step = Termination()

game_research_sm.add_steps([
    entry_step,
    query_step,
    evaluate_step,
    web_search_step,
    answer_step,
    termination_step
])

# Connect the steps
game_research_sm.connect(entry_step, query_step)
game_research_sm.connect(query_step, evaluate_step)
game_research_sm.connect(evaluate_step, [web_search_step, answer_step], condition=should_search_web)
game_research_sm.connect(web_search_step, answer_step)
game_research_sm.connect(answer_step, termination_step)

print("✅ StateMachine created successfully!")

✅ StateMachine created successfully!


In [22]:
# Test queries for the workflow appraoch here is the final answer for all the questions

test_queries = [
    "When were Pokémon Gold and Silver released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for PlayStation 5?"
]

print("STarting StateMachine Agent")

for i, query in enumerate(test_queries, 1):
    print(f"\nQuery {i}: {query}")
    
    try:
        # Initialize state for StateMachine
        initial_state = GameResearchState(
            user_query=query,
            retrieved_docs="",
            evaluation_result="",
            web_search_result="",
            final_answer="",
            confidence_score=0.0,
            sources_used=[]
        )
        
        # Run the StateMachine workflow
        run_result = game_research_sm.run(initial_state)
        final_state = run_result.get_final_state()
        
        print(f"Final Answer:")
        print(f"{final_state['final_answer']}")
        print(f"Sources used: {', '.join(final_state['sources_used'])}")
        print(f"Confidence: {final_state['confidence_score']:.1%}")
        
    except Exception as e:
        print(f"Error: {e}")

STarting StateMachine Agent

Query 1: When were Pokémon Gold and Silver released?
[StateMachine] Starting: __entry__
Searcing my database for information about the question :)
Found documents
[StateMachine] Executing step: query_db
 EValuating the quality of the answer from the db. Hope its GOOD
Evaluation complete. Confidence: 0.8
[StateMachine] Executing step: evaluate
Generating my final answer...
Final answer generated
[StateMachine] Executing step: generate_answer
[StateMachine] Terminating: __termination__
Final Answer:
Pokémon Gold and Silver were released in 1999 for the Game Boy Color. These games are notable for being the second generation of Pokémon games, introducing new regions, Pokémon, and gameplay mechanics. If you have any more questions about Pokémon or other games, feel free to ask!
Sources used: vector_database
Confidence: 80.0%

Query 2: Which one was the first 3D platformer Mario game?
[StateMachine] Starting: __entry__
Searcing my database for information about t

In [ ]:
# so it works well and so sad that they did not released Mortal Kombat X for PS5 :(( But at least it works.
# Thanks for this project was super nice serving as a beta tester :)


### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes


# since timing for beta testing is really tught, i will do this after submittingh my other projects promise!